<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/SPOT_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import modules
!pip install rasterio
import rasterio
import ee
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 37.9 MB/s eta 0:00:00


In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

In [3]:


def save_df_to_drive(df, file_path_in_drive):
    """
    Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a Google Drive file path

    parameters:
      df (pd.DataFrame): The Pandas DataFrame needing to be saved
      file_path_in_drive (str): Google Drive folder file path that will store the CSV file
    """
    try:
        # Ensure the destination directory exists
        drive.mount('/content/drive')  # Mount Google Drive to access files

        # Get the absolute path in Google Drive
        absolute_path_in_drive = os.path.join('/content/drive/My Drive', file_path_in_drive)

        # Ensure the destination directory exists in Google Drive
        os.makedirs(os.path.dirname(absolute_path_in_drive), exist_ok=True)

        # Save the DataFrame to the specified file path in Google Drive
        df.to_csv(absolute_path_in_drive, index=False)

        print(f"DataFrame saved to Google Drive at '{absolute_path_in_drive}'")

    except Exception as e:
        print("An error occurred:", str(e))

# Example usage:
# Assuming your file_path_in_drive is 'YourFolder/YourFile.csv'
# Make sure to change it according to your actual folder structure
#save_df_to_drive(your_dataframe, 'YourFolder/YourFile.csv')


Get SPOT-7 Image from GEE

In [ ]:
# Define the image
#image = ee.Image('projects/mapwaps-sabiecroc/assets/SPOT7')

In [4]:
#visualize the SPOT-7 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['b1', 'b2', 'b3'],
    'min': 200,
    'max': 800,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('projects/mapwaps-sabiecroc/assets/SPOT7')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

In [7]:
image = ee.Image('projects/mapwaps-sabiecroc/assets/SPOT7')
# Define vegetation indicies

GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('b2'),
    'RED': image.select('b1'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('b3'),
    'Red': image.select('b1'),
}).rename('WBI')


# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [3072, 3584], 'crs': 'EPSG:4326', 'crs_transform': [5.555555555555556e-05, 0, 30.880333333333333, 0, -5.555555555555556e-05, -25.077277777777777]}, {'id': 'b2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [3072, 3584], 'crs': 'EPSG:4326', 'crs_transform': [5.555555555555556e-05, 0, 30.880333333333333, 0, -5.555555555555556e-05, -25.077277777777777]}, {'id': 'b3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [3072, 3584], 'crs': 'EPSG:4326', 'crs_transform': [5.555555555555556e-05, 0, 30.880333333333333, 0, -5.555555555555556e-05, -25.077277777777777]}, {'id': 'b4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [3072, 3584], 'crs': 'EPSG:4326', 'crs_transform': [5.555555555555556e-05, 0, 30.880333333333333, 0,

In [8]:
# Load the training data (e.g., a FeatureCollection) from GEE
training_data = ee.FeatureCollection('projects/mapwaps-sabiecroc/assets/Training_setSc')

# Convert the training data to a GeoDataFrame
features = training_data.getInfo()['features']
training_subset = gpd.GeoDataFrame.from_features(features)

# Display the GeoDataFrame
print(training_subset.head())

                     geometry DIRECTION DIRECTIO_1      DateTime DateTime_1  \
0  POINT (30.83475 -25.53864)                      -2.209133e+12              
1  POINT (30.83556 -25.53828)                      -2.209133e+12              
2  POINT (30.94377 -25.06340)         E            -2.209133e+12              
3  POINT (30.93543 -25.06455)         E            -2.209133e+12              
4  POINT (30.97520 -24.95397)         E             1.691478e+12              

   Distance_1  Distance_t Done Done_1  FID_  ...    Val_id Val_id_1  \
0           0           0                 0  ...  Val_4440            
1           0           0                 0  ...  Val_4441            
2           0           5    Y            0  ...    Val_19            
3           0          30    Y            0  ...    Val_23            
4           0          15    Y            0  ...    Val_33            

   Validation          X          Y        Z  Z_1  coords_x1 coords_x2  \
0           0   0.000000

In [9]:
#select bands
bands = ['b1', 'b2', 'b3', 'b4', 'GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI']

# Sample regions from the image at training feature locations
training_subset = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

In [10]:
"""Classification using random forest"""

#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_subset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

In [11]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m

Map(center=[-25.17681845787754, 30.965666655952784], controls=(WidgetControl(options=['position', 'transparent…